In [ ]:
!pip install psycopg2 neo4j pandas


In [8]:
import psycopg2
import pandas as pd

# PostgreSQL Connection
pg_conn = psycopg2.connect(
    database="amazon_db",
    user="postgres",
    password="******",
    host="localhost",
    port="5432"
)
pg_cursor = pg_conn.cursor()

# Test Query
pg_cursor.execute("SELECT COUNT(*) FROM amazon_products;")
result = pg_cursor.fetchone()

print("✅ PostgreSQL is connected! Total products:", result[0])


✅ PostgreSQL is connected! Total products: 548552


In [9]:
from neo4j import GraphDatabase

# Neo4j Connection
neo4j_driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "*******"))

def test_neo4j_connection():
    query = "MATCH (p:Product) RETURN count(p) AS total_products"
    with neo4j_driver.session() as session:
        result = session.run(query)
        total_products = result.single()["total_products"]
        print("✅ Neo4j is connected! Total products:", total_products)

test_neo4j_connection()

✅ Neo4j is connected! Total products: 548552


<h3>First Attempt -- only work when searching with exact title</h3>

In [10]:
def get_recommendations(product_title):
    print(f"🔍 Searching for recommendations for: {product_title}")

    # Step 1: Query Neo4j for related products
    query = """
    MATCH (p:Product {title: $title})-[:CO_PURCHASED_WITH]->(p2)
    RETURN p2.asin AS recommended_asin
    ORDER BY p2.sales_rank
    LIMIT 5
    """
    with neo4j_driver.session() as session:
        result = session.run(query, title=product_title)
        asin_list = [record["recommended_asin"] for record in result]

    print("🔍 ASINs returned from Neo4j:", asin_list)  # Debugging step

    if not asin_list:
        return "⚠ No recommendations found."

    # Step 2: Query PostgreSQL for product details
    sql_query = """
    SELECT asin, title, product_group, sales_rank
    FROM amazon_products
    WHERE asin IN %s
    """
    pg_cursor.execute(sql_query, (tuple(asin_list),))
    recommended_products = pg_cursor.fetchall()

    if not recommended_products:
        return "⚠ PostgreSQL did not return any product details."

    print("✅ PostgreSQL returned product details!")  # Debugging step

    return pd.DataFrame(recommended_products, columns=["ASIN", "Title", "Product Group", "Sales Rank"])

# Example Usage
product_to_recommend = "Your Five- and Six-Year-Old: As They Grow"  # Use an existing product title
recommendations = get_recommendations(product_to_recommend)

from IPython.display import display
display(recommendations)


🔍 Searching for recommendations for: Your Five- and Six-Year-Old: As They Grow
🔍 ASINs returned from Neo4j: ['0761521364', '0060922761', '0312264208', '0440506735', '0440506743']
✅ PostgreSQL returned product details!


,ASIN,Title,Product Group,Sales Rank
0,0060922761,Child Behavior: The Classic Childcare Manual f...,Book,5006
1,0312264208,Your Three- and Four-Year-Old: As They Grow,Book,16161
2,0440506735,Your Five Year Old: Sunny and Serene,Book,20389
3,0440506743,Your Six-Year-Old : Loving and Defiant,Book,23158
4,0761521364,Setting Limits with Your Strong-Willed Child :...,Book,333


<h3>Second Attempt </h3>

In [11]:
def get_recommendations(search_input, search_by_asin=False):
    """
    Get product recommendations based on:
    - Keywords in the title (default)
    - ASIN (if search_by_asin=True)
    
    Recommendations are ranked by **average rating** from PostgreSQL.
    """
    if search_by_asin:
        print(f"🔍 Searching recommendations for ASIN: {search_input}")
        query = """
        MATCH (p:Product {asin: $asin})-[:CO_PURCHASED_WITH]->(p2)
        RETURN p2.asin AS recommended_asin
        ORDER BY p2.sales_rank
        LIMIT 5
        """
    else:
        print(f"🔍 Searching recommendations for title containing: '{search_input}'")
        query = """
        MATCH (p:Product)-[:CO_PURCHASED_WITH]->(p2)
        WHERE toLower(p.title) CONTAINS toLower($title)
        RETURN p2.asin AS recommended_asin
        ORDER BY p2.sales_rank
        LIMIT 5
        """

    with neo4j_driver.session() as session:
        print("🟡 Running Neo4j Query...")
        result = session.run(query, {"title": search_input} if not search_by_asin else {"asin": search_input})

        asin_list = [record["recommended_asin"] for record in result]
        print("🔍 ASINs returned from Neo4j:", asin_list)

    if not asin_list:
        return "⚠ No recommendations found in Neo4j."

    # Step 2: Query PostgreSQL for product details, ranked by avg rating
    sql_query = """
    SELECT p.asin, p.title, p.product_group, p.sales_rank, 
           COALESCE((SELECT AVG(r.rating) FROM amazon_reviews r WHERE r.asin = p.asin), 0) AS avg_rating
    FROM amazon_products p
    WHERE p.asin IN %s
    ORDER BY avg_rating DESC
    """
    pg_cursor.execute(sql_query, (tuple(asin_list),))
    recommended_products = pg_cursor.fetchall()

    if not recommended_products:
        return "⚠ PostgreSQL did not return any product details."

    print("✅ PostgreSQL returned product details!")

    return pd.DataFrame(recommended_products, columns=["ASIN", "Title", "Product Group", "Sales Rank", "Avg Rating"])


In [12]:
# Example Usage: Search by Keyword
recommendations_by_title = get_recommendations("six-year")
from IPython.display import display
display(recommendations_by_title)

# Example Usage: Search by ASIN
recommendations_by_asin = get_recommendations("0761521364", search_by_asin=True)
display(recommendations_by_asin)

🔍 Searching recommendations for title containing: 'six-year'
🟡 Running Neo4j Query...
🔍 ASINs returned from Neo4j: ['0380811960', '0761521364', '0761521364', '0060923288', '0060922761']
✅ PostgreSQL returned product details!


,ASIN,Title,Product Group,Sales Rank,Avg Rating
0,0060922761,Child Behavior: The Classic Childcare Manual f...,Book,5006,5.0000000000000000
1,0761521364,Setting Limits with Your Strong-Willed Child :...,Book,333,4.8666666666666667
2,0060923288,Raising Your Spirited Child: A Guide for Paren...,Book,649,4.6386554621848739
3,0380811960,How to Talk So Kids Will Listen & Listen So Ki...,Book,275,4.6122448979591837


🔍 Searching recommendations for ASIN: 0761521364
🟡 Running Neo4j Query...
🔍 ASINs returned from Neo4j: ['0380811960', '0060923288', '076112182X', '0071383018']
✅ PostgreSQL returned product details!


,ASIN,Title,Product Group,Sales Rank,Avg Rating
0,0060923288,Raising Your Spirited Child: A Guide for Paren...,Book,649,4.6386554621848739
1,0380811960,How to Talk So Kids Will Listen & Listen So Ki...,Book,275,4.6122448979591837
2,076112182X,The Pocket Parent,Book,945,4.5000000000000000
3,0071383018,"Parenting the Strong-Willed Child, Revised and...",Book,2480,4.3333333333333333


<h3>Final Attempt -- Only Uses Neo4j for Recommendations/Fetches Ratings from PostgreSQL/Orders Results by Rating/Does Not Use Category-Based Recommendations </h3>

In [13]:
def get_recommendations(search_input, search_by_asin=False):
    """
    Get product recommendations based on:
    - Keywords in the title (default)
    - ASIN (if search_by_asin=True)
    
    Recommendations are ranked by **average rating** from PostgreSQL.
    """
    if search_by_asin:
        print(f"🔍 Searching recommendations for ASIN: {search_input}")
        query = """
        MATCH (p:Product {asin: $asin})-[:CO_PURCHASED_WITH]->(p2)
        RETURN DISTINCT p2.asin AS recommended_asin, p2.sales_rank
        ORDER BY p2.sales_rank
        LIMIT 10
        """
    else:
        print(f"🔍 Searching recommendations for title containing: '{search_input}'")
        query = """
        MATCH (p:Product)-[:CO_PURCHASED_WITH]->(p2)
        WHERE toLower(p.title) CONTAINS toLower($title)
        RETURN DISTINCT p2.asin AS recommended_asin, p2.sales_rank
        ORDER BY p2.sales_rank
        LIMIT 10
        """

    with neo4j_driver.session() as session:
        print("🟡 Running Neo4j Query...")
        result = session.run(query, {"title": search_input} if not search_by_asin else {"asin": search_input})

        # Extract ASINs and remove duplicates
        asin_list = list(set(record["recommended_asin"] for record in result))
        print("🔍 ASINs returned from Neo4j:", asin_list)

    if not asin_list:
        return "⚠ No recommendations found in Neo4j."

    # Step 2: Query PostgreSQL for product details, ranked by avg rating
    sql_query = """
    SELECT p.asin, p.title, p.product_group, p.sales_rank, 
           COALESCE((SELECT AVG(r.rating) FROM amazon_reviews r WHERE r.asin = p.asin), 0) AS avg_rating
    FROM amazon_products p
    WHERE p.asin IN %s
    ORDER BY avg_rating DESC
    """
    pg_cursor.execute(sql_query, (tuple(asin_list),))
    recommended_products = pg_cursor.fetchall()

    if not recommended_products:
        return "⚠ PostgreSQL did not return any product details."

    print("✅ PostgreSQL returned product details!")

    return pd.DataFrame(recommended_products, columns=["ASIN", "Title", "Product Group", "Sales Rank", "Avg Rating"])

In [14]:
# Example Usage: Search by Keyword
recommendations_by_title = get_recommendations("six-year")
from IPython.display import display
display(recommendations_by_title)

# Example Usage: Search by ASIN
recommendations_by_asin = get_recommendations("0761521364", search_by_asin=True)
display(recommendations_by_asin)

🔍 Searching recommendations for title containing: 'six-year'
🟡 Running Neo4j Query...
🔍 ASINs returned from Neo4j: ['0471346985', '0440506751', '0316777153', '0761521364', '0380811960', '0060923288', '0440506816', '0440506506', '0440506492', '0060922761']
✅ PostgreSQL returned product details!


,ASIN,Title,Product Group,Sales Rank,Avg Rating
0,0060922761,Child Behavior: The Classic Childcare Manual f...,Book,5006,5.0000000000000000
1,0761521364,Setting Limits with Your Strong-Willed Child :...,Book,333,4.8666666666666667
2,0060923288,Raising Your Spirited Child: A Guide for Paren...,Book,649,4.6386554621848739
3,0380811960,How to Talk So Kids Will Listen & Listen So Ki...,Book,275,4.6122448979591837
4,0316777153,The Family Nutrition Book: Everything You Need...,Book,15531,4.5000000000000000
5,0440506816,Your Eight Year Old : Lively and Outgoing,Book,14686,4.2857142857142857
6,0440506506,Your Seven-Year-Old : Life in a Minor Key,Book,15437,4.2500000000000000
7,0471346985,Quick Meals for Healthy Kids and Busy Parents ...,Book,7073,3.6000000000000000
8,0440506751,Your Four-Year-Old : Wild and Wonderful,Book,14352,3.5714285714285714
9,0440506492,Your Three-Year-Old : Friend or Enemy,Book,9540,3.0000000000000000


🔍 Searching recommendations for ASIN: 0761521364
🟡 Running Neo4j Query...
🔍 ASINs returned from Neo4j: ['0380811960', '076112182X', '0071383018', '0060923288']
✅ PostgreSQL returned product details!


,ASIN,Title,Product Group,Sales Rank,Avg Rating
0,0060923288,Raising Your Spirited Child: A Guide for Paren...,Book,649,4.6386554621848739
1,0380811960,How to Talk So Kids Will Listen & Listen So Ki...,Book,275,4.6122448979591837
2,076112182X,The Pocket Parent,Book,945,4.5000000000000000
3,0071383018,"Parenting the Strong-Willed Child, Revised and...",Book,2480,4.3333333333333333


<h3>Final Attempt V2 -- Combines Neo4j (Co-Purchase) + PostgreSQL (Category-Based)/ Fetches Ratings from PostgreSQL /Orders Results by Rating & Sales Rank</h3>

In [15]:
import pandas as pd

def get_combined_recommendations(search_input, search_by_asin=False):
    """
    Get product recommendations based on:
    - Co-purchase graph relationships (Neo4j)
    - Category-based recommendations (PostgreSQL)
    - Rank by **average rating**
    """

    ### Step 1: Neo4j Query (Co-Purchase Recommendations) ###
    if search_by_asin:
        print(f"🔍 Searching recommendations for ASIN: {search_input}")
        neo4j_query = """
        MATCH (p:Product {asin: $asin})-[:CO_PURCHASED_WITH]->(p2)
        RETURN DISTINCT p2.asin AS recommended_asin
        LIMIT 10
        """
    else:
        print(f"🔍 Searching recommendations for title containing: '{search_input}'")
        neo4j_query = """
        MATCH (p:Product)-[:CO_PURCHASED_WITH]->(p2)
        WHERE toLower(p.title) CONTAINS toLower($title)
        RETURN DISTINCT p2.asin AS recommended_asin
        LIMIT 10
        """

    with neo4j_driver.session() as session:
        print("🟡 Running Neo4j Query...")
        result = session.run(neo4j_query, {"title": search_input} if not search_by_asin else {"asin": search_input})

        # Extract ASINs from Neo4j
        neo4j_asins = list(set(record["recommended_asin"] for record in result))
        print("🔍 ASINs returned from Neo4j:", neo4j_asins)

    ### Step 2: PostgreSQL Query (Category-Based Recommendations + Ratings) ###
    if search_by_asin:
        category_query = """
        SELECT DISTINCT p2.asin
        FROM amazon_products p1
        JOIN amazon_products p2 ON p1.product_group = p2.product_group
        WHERE p1.asin = %s AND p1.asin <> p2.asin
        LIMIT 10
        """
    else:
        category_query = """
        SELECT DISTINCT p2.asin
        FROM amazon_products p1
        JOIN amazon_products p2 ON p1.product_group = p2.product_group
        WHERE p1.title ILIKE %s AND p1.asin <> p2.asin
        LIMIT 10
        """

    pg_cursor.execute(category_query, (search_input if search_by_asin else f"%{search_input}%",))
    category_asins = [row[0] for row in pg_cursor.fetchall()]
    print("🔍 ASINs returned from PostgreSQL (Category-based):", category_asins)

    ### Step 3: Combine Neo4j + PostgreSQL ASINs ###
    combined_asins = list(set(neo4j_asins + category_asins))
    if not combined_asins:
        return "⚠ No recommendations found in Neo4j or PostgreSQL."

    ### Step 4: Get Full Product Details & Rank by Avg Rating ###
    sql_query = """
    SELECT p.asin, p.title, p.product_group, p.sales_rank, 
           COALESCE((SELECT AVG(r.rating) FROM amazon_reviews r WHERE r.asin = p.asin), 0) AS avg_rating
    FROM amazon_products p
    WHERE p.asin IN %s
    ORDER BY avg_rating DESC, p.sales_rank ASC
    """
    pg_cursor.execute(sql_query, (tuple(combined_asins),))
    recommended_products = pg_cursor.fetchall()

    if not recommended_products:
        return "⚠ PostgreSQL did not return any product details."

    print("✅ PostgreSQL returned product details!")

    return pd.DataFrame(recommended_products, columns=["ASIN", "Title", "Product Group", "Sales Rank", "Avg Rating"])


# Example Usage (Title Search)
recommendations = get_combined_recommendations("six-year")
from IPython.display import display
display(recommendations)

# Example Usage (ASIN Search)
recommendations_asin = get_combined_recommendations("0761521364", search_by_asin=True)
display(recommendations_asin)

🔍 Searching recommendations for title containing: 'six-year'
🟡 Running Neo4j Query...
🔍 ASINs returned from Neo4j: ['0440506751', '0440506735', '0761521364', '0380811960', '0440506506', '0440506816', '0440506743', '0440506492', '0060922761', '0312264208']
🔍 ASINs returned from PostgreSQL (Category-based): ['0000037931', '0001047655', '0001053388', '0001053736', '0001053744', '0001054600', '0001056298', '000105709X', '0001057170', '0001360817']
✅ PostgreSQL returned product details!


,ASIN,Title,Product Group,Sales Rank,Avg Rating
0,0060922761,Child Behavior: The Classic Childcare Manual f...,Book,5006,5.0000000000000000
1,0312264208,Your Three- and Four-Year-Old: As They Grow,Book,16161,5.0000000000000000
2,0001053744,Pearl and Sir Orfeo,Book,500890,5.0000000000000000
3,0761521364,Setting Limits with Your Strong-Willed Child :...,Book,333,4.8666666666666667
4,0380811960,How to Talk So Kids Will Listen & Listen So Ki...,Book,275,4.6122448979591837
5,0001053736,Sir Gawain and the Green Knight,Book,53150,4.6000000000000000
6,0440506816,Your Eight Year Old : Lively and Outgoing,Book,14686,4.2857142857142857
7,0440506506,Your Seven-Year-Old : Life in a Minor Key,Book,15437,4.2500000000000000
8,0440506743,Your Six-Year-Old : Loving and Defiant,Book,23158,4.2000000000000000
9,0001054600,The Picture of Dorian Gray,Book,2449141,4.1595744680851064


🔍 Searching recommendations for ASIN: 0761521364
🟡 Running Neo4j Query...
🔍 ASINs returned from Neo4j: ['0380811960', '076112182X', '0071383018', '0060923288']
🔍 ASINs returned from PostgreSQL (Category-based): ['0000037931', '0001047655', '0001053388', '0001053736', '0001053744', '0001054600', '0001056298', '000105709X', '0001057170', '0001360817']
✅ PostgreSQL returned product details!


,ASIN,Title,Product Group,Sales Rank,Avg Rating
0,0001053744,Pearl and Sir Orfeo,Book,500890,5.0000000000000000
1,0060923288,Raising Your Spirited Child: A Guide for Paren...,Book,649,4.6386554621848739
2,0380811960,How to Talk So Kids Will Listen & Listen So Ki...,Book,275,4.6122448979591837
3,0001053736,Sir Gawain and the Green Knight,Book,53150,4.6000000000000000
4,076112182X,The Pocket Parent,Book,945,4.5000000000000000
5,0071383018,"Parenting the Strong-Willed Child, Revised and...",Book,2480,4.3333333333333333
6,0001054600,The Picture of Dorian Gray,Book,2449141,4.1595744680851064
7,0001047655,Prodigal Daughter,Book,1116690,3.7666666666666667
8,0001057170,CLASSIC CONNOLLY BOXED SET,Book,708137,3.5000000000000000
9,0001056298,Select Short Stor Jeffery Arch,Book,1507473,0


With Community Detection

In [16]:
def fetch_product_details(pg_cursor, asins):
    if not asins:
        return []
    query = """
    SELECT 
        p.asin, 
        p.title, 
        p.product_group, 
        p.sales_rank,
        COALESCE(AVG(r.rating), 0) AS avg_rating
    FROM 
        amazon_products p
    LEFT JOIN 
        amazon_reviews r ON p.asin = r.asin
    WHERE 
        p.asin IN %s
    GROUP BY 
        p.asin, p.title, p.product_group, p.sales_rank
    ORDER BY 
        p.sales_rank ASC
    """
    pg_cursor.execute(query, (tuple(asins),))
    return pg_cursor.fetchall()

In [17]:
def get_recommendations(search_input, search_by_asin, pg_cursor, neo4j_driver):
    import pandas as pd
    
    if not search_by_asin:
        # Title Search
        # Step 1: Find matching ASINs
        matching_query = """
        SELECT asin FROM amazon_products 
        WHERE title ILIKE %s 
        ORDER BY sales_rank ASC 
        LIMIT 3
        """
        pg_cursor.execute(matching_query, (f"%{search_input}%",))
        matching_asins = [row[0] for row in pg_cursor.fetchall()]

        if not matching_asins:
            return "⚠ No matching products found."

        # Step 2: Get recommendation ASINs from Neo4j
        neo4j_query = """
        MATCH (p:Product)-[:CO_PURCHASED_WITH]->(p2)
        WHERE p.asin IN $matching_asins AND p.louvainCommunity = p2.louvainCommunity
        RETURN DISTINCT p2.asin AS recommended_asin
        """
        with neo4j_driver.session() as session:
            result = session.run(neo4j_query, matching_asins=matching_asins)
            neo4j_recommendations = [record["recommended_asin"] for record in result 
                                    if record["recommended_asin"] not in matching_asins]

        # Step 3: Fetch all products in one query
        all_asins = matching_asins + neo4j_recommendations
        all_products = fetch_product_details(pg_cursor, all_asins)

        # Step 4: Split results into matching and recommendations
        matching_products = [p for p in all_products if p[0] in matching_asins]
        recommendations_products = [p for p in all_products if p[0] not in matching_asins][:7]

        # Step 5: Combine and format
        final_products = matching_products + recommendations_products
        final_df = pd.DataFrame(final_products, 
                              columns=["ASIN", "Title", "Product Group", "Sales Rank", "Avg Rating"]).head(10)

    else:
        # ASIN Search
        # Step 1: Verify the ASIN exists
        product_query = """
        SELECT asin FROM amazon_products WHERE asin = %s
        """
        pg_cursor.execute(product_query, (search_input,))
        product_asin = pg_cursor.fetchone()
        if not product_asin:
            return "⚠ Product not found."

        # Step 2: Get recommendation ASINs from Neo4j
        neo4j_query = """
        MATCH (p:Product {asin: $asin})-[:CO_PURCHASED_WITH]->(p2)
        WHERE p.louvainCommunity = p2.louvainCommunity
        RETURN DISTINCT p2.asin AS recommended_asin
        """
        with neo4j_driver.session() as session:
            result = session.run(neo4j_query, asin=search_input)
            neo4j_recommendations = [record["recommended_asin"] for record in result 
                                    if record["recommended_asin"] != search_input]

        # Step 3: Fetch all products in one query
        all_asins = [search_input] + neo4j_recommendations
        all_products = fetch_product_details(pg_cursor, all_asins)

        # Step 4: Split into specific product and recommendations
        product = next(p for p in all_products if p[0] == search_input)
        recommendations_products = [p for p in all_products if p[0] != search_input][:9]

        # Step 5: Combine and format
        final_products = [product] + recommendations_products
        final_df = pd.DataFrame(final_products, 
                              columns=["ASIN", "Title", "Product Group", "Sales Rank", "Avg Rating"]).head(10)

    print("✅ Recommendations generated!")
    return final_df

In [18]:
# Example usage:
get_recommendations("the great gatsby", False, pg_cursor, neo4j_driver)


✅ Recommendations generated!


,ASIN,Title,Product Group,Sales Rank,Avg Rating
0,0684801523,The Great Gatsby,Book,956,4.1809421841541756
1,6301247485,The Great Gatsby,Video,2851,3.8000000000000000
2,0764586017,Fitzgerald's The Great Gatsby (Cliffs Notes),Book,4163,3.0000000000000000
3,0316769487,The Catcher in the Rye,Book,60,4.1683593750000000
4,0142000663,The Grapes of Wrath : (Centennial Edition),Book,201,4.2920696324951644
5,0553210092,The Scarlet Letter,Book,368,3.5143678160919540
6,0553210793,The Adventures of Huckleberry Finn (Bantam Cla...,Book,1138,3.9966555183946488
7,B00007KQA4,Of Mice And Men (Special Edition),DVD,4915,4.5760869565217391
8,0764586041,The Adventures of Huckleberry Finn (Cliffs Notes),Book,9206,4.3333333333333333
9,0764585886,The Crucible (Cliffs Notes),Book,18300,3.5454545454545455


With Promotion

In [ ]:
!pip install psycopg2-binary pymongo redis

In [20]:
import pandas as pd
import pymongo
import redis
import json

def fetch_product_details(pg_cursor, asins):
    if not asins:
        return []
    query = """
    SELECT 
        p.asin, 
        p.title, 
        p.product_group, 
        p.sales_rank,
        COALESCE(AVG(r.rating), 0) AS avg_rating
    FROM 
        amazon_products p
    LEFT JOIN 
        amazon_reviews r ON p.asin = r.asin
    WHERE 
        p.asin IN %s
    GROUP BY 
        p.asin, p.title, p.product_group, p.sales_rank
    ORDER BY 
        p.sales_rank ASC
    """
    pg_cursor.execute(query, (tuple(asins),))
    return pg_cursor.fetchall()

def get_recommendations(search_input, search_by_asin, pg_cursor, neo4j_driver):
    # Connect to MongoDB and Redis
    mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
    db = mongo_client["amazon"]
    promoted_collection = db["promoted_products"]
    r = redis.Redis(host='localhost', port=6379, db=0)

    # Define function to fetch promoted products
    def get_promoted_products():
        try:
            # Check Redis cache first
            cached_data = r.keys("promoted:*")
            if cached_data:
                promoted = [json.loads(r.get(k).decode()) for k in cached_data]
                return promoted[:3]
            print("Fetching from MongoDB...")
            # Fetch from MongoDB if not in cache
            promoted = list(promoted_collection.find().limit(3))
            # Cache in Redis with 7-day expiration
            for p in promoted:
                p["_id"] = str(p["_id"])
                r.set(f"promoted:{p['asin']}", json.dumps(p), ex=604800)  # 604800 seconds = 7 days
            return promoted
        except Exception as e:
            print(f"Error fetching promoted products: {e}")
            return []

    # Generate main recommendations
    if not search_by_asin:
        # Title Search
        # Step 1: Find matching ASINs
        matching_query = """
        SELECT asin FROM amazon_products 
        WHERE title ILIKE %s 
        ORDER BY sales_rank ASC 
        LIMIT 3
        """
        pg_cursor.execute(matching_query, (f"%{search_input}%",))
        matching_asins = [row[0] for row in pg_cursor.fetchall()]

        if not matching_asins:
            return "⚠ No matching products found."

        # Step 2: Get recommendation ASINs from Neo4j
        neo4j_query = """
        MATCH (p:Product)-[:CO_PURCHASED_WITH]->(p2)
        WHERE p.asin IN $matching_asins AND p.louvainCommunity = p2.louvainCommunity
        RETURN DISTINCT p2.asin AS recommended_asin
        """
        with neo4j_driver.session() as session:
            result = session.run(neo4j_query, matching_asins=matching_asins)
            neo4j_recommendations = [record["recommended_asin"] for record in result 
                                    if record["recommended_asin"] not in matching_asins]

        # Step 3: Fetch all products in one query
        all_asins = matching_asins + neo4j_recommendations
        all_products = fetch_product_details(pg_cursor, all_asins)

        # Step 4: Split results into matching and recommendations
        matching_products = [p for p in all_products if p[0] in matching_asins]
        recommendations_products = [p for p in all_products if p[0] not in matching_asins][:7]

        # Step 5: Create main recommendations DataFrame
        main_products = matching_products + recommendations_products
        main_df = pd.DataFrame(main_products, 
                              columns=["ASIN", "Title", "Product Group", "Sales Rank", "Avg Rating"])
        main_df['is_promoted'] = False

    else:
        # ASIN Search
        # Step 1: Verify the ASIN exists
        product_query = """
        SELECT asin FROM amazon_products WHERE asin = %s
        """
        pg_cursor.execute(product_query, (search_input,))
        product_asin = pg_cursor.fetchone()
        if not product_asin:
            return "⚠ Product not found."

        # Step 2: Get recommendation ASINs from Neo4j
        neo4j_query = """
        MATCH (p:Product {asin: $asin})-[:CO_PURCHASED_WITH]->(p2)
        WHERE p.louvainCommunity = p2.louvainCommunity
        RETURN DISTINCT p2.asin AS recommended_asin
        """
        with neo4j_driver.session() as session:
            result = session.run(neo4j_query, asin=search_input)
            neo4j_recommendations = [record["recommended_asin"] for record in result 
                                    if record["recommended_asin"] != search_input]

        # Step 3: Fetch all products in one query
        all_asins = [search_input] + neo4j_recommendations
        all_products = fetch_product_details(pg_cursor, all_asins)

        # Step 4: Split into specific product and recommendations
        product = next(p for p in all_products if p[0] == search_input)
        recommendations_products = [p for p in all_products if p[0] != search_input][:9]

        # Step 5: Create main recommendations DataFrame
        main_products = [product] + recommendations_products
        main_df = pd.DataFrame(main_products, 
                              columns=["ASIN", "Title", "Product Group", "Sales Rank", "Avg Rating"])
        main_df['is_promoted'] = False

    # Add promoted products
    promoted = get_promoted_products()
    promoted_asins = [p['asin'] for p in promoted]
    promoted_details = fetch_product_details(pg_cursor, promoted_asins)
    promoted_df = pd.DataFrame(promoted_details, 
                              columns=["ASIN", "Title", "Product Group", "Sales Rank", "Avg Rating"])
    promoted_df['is_promoted'] = True

    # Combine main recommendations and promoted products
    final_df = pd.concat([main_df, promoted_df], ignore_index=True)

    print("✅ Recommendations generated with promoted products!")
    return final_df



In [21]:
get_recommendations("the great gatsby", False, pg_cursor, neo4j_driver)

✅ Recommendations generated with promoted products!


,ASIN,Title,Product Group,Sales Rank,Avg Rating,is_promoted
0,0684801523,The Great Gatsby,Book,956,4.1809421841541756,False
1,6301247485,The Great Gatsby,Video,2851,3.8000000000000000,False
2,0764586017,Fitzgerald's The Great Gatsby (Cliffs Notes),Book,4163,3.0000000000000000,False
3,0316769487,The Catcher in the Rye,Book,60,4.1683593750000000,False
4,0142000663,The Grapes of Wrath : (Centennial Edition),Book,201,4.2920696324951644,False
5,0553210092,The Scarlet Letter,Book,368,3.5143678160919540,False
6,0553210793,The Adventures of Huckleberry Finn (Bantam Cla...,Book,1138,3.9966555183946488,False
7,B00007KQA4,Of Mice And Men (Special Edition),DVD,4915,4.5760869565217391,False
8,0764586041,The Adventures of Huckleberry Finn (Cliffs Notes),Book,9206,4.3333333333333333,False
9,0764585886,The Crucible (Cliffs Notes),Book,18300,3.5454545454545455,False


In [22]:
get_recommendations("0684801523", True, pg_cursor, neo4j_driver)

✅ Recommendations generated with promoted products!


,ASIN,Title,Product Group,Sales Rank,Avg Rating,is_promoted
0,0684801523,The Great Gatsby,Book,956,4.1809421841541756,False
1,0316769487,The Catcher in the Rye,Book,60,4.1683593750000000,False
2,0142000663,The Grapes of Wrath : (Centennial Edition),Book,201,4.2920696324951644,False
3,0553210092,The Scarlet Letter,Book,368,3.5143678160919540,False
4,0553210793,The Adventures of Huckleberry Finn (Bantam Cla...,Book,1138,3.9966555183946488,False
5,0764586017,Fitzgerald's The Great Gatsby (Cliffs Notes),Book,4163,3.0000000000000000,False
6,B00005LMOC,Lorca,Music,203959,4.2500000000000000,True
7,0968677614,MENagerie: Stories of Passion and Dark Fantasy,Book,258231,5.0000000000000000,True
8,0316174688,"Flight (Smallville Series for Young Adults, No...",Book,330922,4.6666666666666667,True


In [23]:
get_recommendations("Boat Racing", False, pg_cursor, neo4j_driver)

✅ Recommendations generated with promoted products!


,ASIN,Title,Product Group,Sales Rank,Avg Rating,is_promoted
0,0070308179,Around the Buoys: A Manual of Sailboat Racing ...,Book,134829,3.5000000000000000,False
1,0393308014,The Tactics of Small Boat Racing,Book,151367,4.6666666666666667,False
2,1898660379,Keelboat & Sportsboat Racing,Book,960464,0,False
3,0805003517,"Sailing Smart : Winning Techniques, Tactics, A...",Book,27752,5.0000000000000000,False
4,0393032965,A Manual of Sail Trim,Book,61567,3.7142857142857143,False
5,0393303330,Advanced Racing Tactics,Book,151015,3.6666666666666667,False
6,0312042787,Championship Tactics : How Anyone Can Sail Fas...,Book,247295,4.2500000000000000,False
7,0924486813,The New Book of Sail Trim,Book,311419,2.6666666666666667,False
8,0071376097,Paul Elvstrom Explains the Racing Rules of Sai...,Book,368845,4.5000000000000000,False
9,B00005LMOC,Lorca,Music,203959,4.2500000000000000,True
